# T-Drive出租车轨迹数据探索分析

本笔记本用于探索和分析T-Drive出租车轨迹数据集，为后续的DBSCAN聚类算法提供数据理解和预处理基础。

## 1. 环境设置

In [ ]:
import sys
import os
from pathlib import Path

# 添加项目根目录到Python路径
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# 导入所需库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体和图表样式
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")
sns.set_palette("husl")

print("Python版本:", sys.version)
print("NumPy版本:", np.__version__)
print("Pandas版本:", pd.__version__)

## 2. 数据加载

In [ ]:
# 导入项目中的数据处理模块
from src.data_processing.loader import TDriveDataLoader, load_tdrive_csv
from src.data_processing.preprocessor import TrajectoryPreprocessor
from src.data_processing.trajectory import Trajectory, TrajectoryPoint

In [ ]:
# 设置数据路径
data_path = "../data/raw"  # 根据实际情况调整

# 创建数据加载器
loader = TDriveDataLoader(data_path, max_workers=2)

# 检查数据文件
import glob
csv_files = glob.glob(f"{data_path}/*.csv")
if not csv_files:
    csv_files = glob.glob(f"{data_path}/*.txt")
    
print(f"找到 {len(csv_files)} 个数据文件:")
for file in csv_files[:5]:  # 显示前5个文件
    print(f"  {os.path.basename(file)}")
if len(csv_files) > 5:
    print(f"  ... 还有 {len(csv_files) - 5} 个文件")

In [ ]:
# 加载单个文件进行初步分析
sample_file = csv_files[0] if csv_files else None

if sample_file:
    print(f"加载样本文件: {os.path.basename(sample_file)}")
    
    # 使用DataFrame加载少量数据进行探索
    df_sample = load_tdrive_csv(sample_file, nrows=10000)
    print(f"加载了 {len(df_sample)} 行数据")
    print("\n数据预览:")
    display(df_sample.head(10))
    
    print("\n数据信息:")
    df_sample.info()
    
    print("\n数据统计描述:")
    display(df_sample.describe())

## 3. 数据质量分析

In [ ]:
# 检查缺失值
if 'df_sample' in locals():
    print("缺失值统计:")
    missing_stats = df_sample.isnull().sum()
    missing_percent = (missing_stats / len(df_sample)) * 100
    
    missing_df = pd.DataFrame({
        '缺失数量': missing_stats,
        '缺失比例(%)': missing_percent.round(2)
    })
    display(missing_df)

In [ ]:
# 检查重复数据
if 'df_sample' in locals():
    duplicates = df_sample.duplicated().sum()
    print(f"重复行数: {duplicates} ({duplicates/len(df_sample)*100:.2f}%)")

In [ ]:
# 检查唯一值
if 'df_sample' in locals():
    print("唯一值统计:")
    unique_stats = {
        '出租车ID数量': df_sample['taxi_id'].nunique(),
        '时间戳唯一值': df_sample['timestamp'].nunique(),
        '纬度唯一值': df_sample['lat'].nunique(),
        '经度唯一值': df_sample['lon'].nunique()
    }
    
    for key, value in unique_stats.items():
        print(f"  {key}: {value}")

## 4. 空间分布分析

In [ ]:
# 绘制空间分布图
if 'df_sample' in locals():
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # 散点图
    ax1 = axes[0]
    scatter = ax1.scatter(df_sample['lon'], df_sample['lat'], 
                         c=range(len(df_sample)), cmap='viridis',
                         s=10, alpha=0.6, edgecolors='none')
    ax1.set_xlabel('经度')
    ax1.set_ylabel('纬度')
    ax1.set_title('轨迹点空间分布')
    plt.colorbar(scatter, ax=ax1, label='点序号')
    
    # 六边形箱图（热力图）
    ax2 = axes[1]
    hb = ax2.hexbin(df_sample['lon'], df_sample['lat'], 
                   gridsize=30, cmap='YlOrRd', alpha=0.8)
    ax2.set_xlabel('经度')
    ax2.set_ylabel('纬度')
    ax2.set_title('轨迹点密度热力图')
    plt.colorbar(hb, ax=ax2, label='点数')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 检查数据是否在北京范围内
if 'df_sample' in locals():
    # 北京的大致边界坐标
    beijing_bounds = {
        'min_lat': 39.4,  # 南边界
        'max_lat': 40.6,  # 北边界
        'min_lon': 115.7,  # 西边界
        'max_lon': 117.4   # 东边界
    }
    
    # 计算在边界内的点比例
    in_beijing = (
        (df_sample['lat'] >= beijing_bounds['min_lat']) & 
        (df_sample['lat'] <= beijing_bounds['max_lat']) &
        (df_sample['lon'] >= beijing_bounds['min_lon']) & 
        (df_sample['lon'] <= beijing_bounds['max_lon'])
    )
    
    in_beijing_percent = in_beijing.mean() * 100
    
    print(f"在北京范围内的点: {in_beijing.sum()} ({in_beijing_percent:.2f}%)")
    print(f"不在北京范围内的点: {(~in_beijing).sum()} ({(100 - in_beijing_percent):.2f}%)")
    
    # 绘制边界检查
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # 绘制所有点
    ax.scatter(df_sample['lon'], df_sample['lat'], 
              c=in_beijing.map({True: 'blue', False: 'red'}),
              s=10, alpha=0.6, edgecolors='none')
    
    # 绘制北京边界
    from matplotlib.patches import Rectangle
    rect = Rectangle((beijing_bounds['min_lon'], beijing_bounds['min_lat']),
                     beijing_bounds['max_lon'] - beijing_bounds['min_lon'],
                     beijing_bounds['max_lat'] - beijing_bounds['min_lat'],
                     fill=False, edgecolor='green', linewidth=2, linestyle='--',
                     label='北京边界')
    ax.add_patch(rect)
    
    ax.set_xlabel('经度')
    ax.set_ylabel('纬度')
    ax.set_title('北京边界检查 (蓝色: 边界内, 红色: 边界外)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 5. 时间特征分析

In [ ]:
# 提取时间特征
if 'df_sample' in locals():
    # 确保时间戳列是datetime类型
    df_sample['timestamp'] = pd.to_datetime(df_sample['timestamp'])
    
    # 提取时间特征
    df_sample['hour'] = df_sample['timestamp'].dt.hour
    df_sample['day'] = df_sample['timestamp'].dt.day
    df_sample['month'] = df_sample['timestamp'].dt.month
    df_sample['weekday'] = df_sample['timestamp'].dt.weekday  # 0=周一, 6=周日
    
    print("时间特征提取完成")
    display(df_sample[['timestamp', 'hour', 'day', 'month', 'weekday']].head())

In [ ]:
# 绘制时间分布
if 'df_sample' in locals():
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 小时分布
    ax1 = axes[0, 0]
    hour_counts = df_sample['hour'].value_counts().sort_index()
    ax1.bar(hour_counts.index, hour_counts.values, color='skyblue', alpha=0.7)
    ax1.set_xlabel('小时')
    ax1.set_ylabel('点数')
    ax1.set_title('按小时分布')
    ax1.set_xticks(range(0, 24, 2))
    
    # 星期分布
    ax2 = axes[0, 1]
    weekday_names = ['周一', '周二', '周三', '周四', '周五', '周六', '周日']
    weekday_counts = df_sample['weekday'].value_counts().sort_index()
    ax2.bar(weekday_counts.index, weekday_counts.values, 
           color='lightgreen', alpha=0.7, tick_label=weekday_names)
    ax2.set_xlabel('星期')
    ax2.set_ylabel('点数')
    ax2.set_title('按星期分布')
    
    # 日分布
    ax3 = axes[1, 0]
    day_counts = df_sample['day'].value_counts().sort_index()
    ax3.plot(day_counts.index, day_counts.values, 'o-', color='salmon', alpha=0.7)
    ax3.set_xlabel('日')
    ax3.set_ylabel('点数')
    ax3.set_title('按日分布')
    
    # 月分布
    ax4 = axes[1, 1]
    month_counts = df_sample['month'].value_counts().sort_index()
    ax4.bar(month_counts.index, month_counts.values, color='gold', alpha=0.7)
    ax4.set_xlabel('月')
    ax4.set_ylabel('点数')
    ax4.set_title('按月分布')
    ax4.set_xticks(range(1, 13))
    
    plt.suptitle('轨迹点时间分布分析', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 6. 出租车个体分析

In [ ]:
# 分析出租车活动
if 'df_sample' in locals():
    # 出租车活动统计
    taxi_stats = df_sample.groupby('taxi_id').agg({
        'timestamp': ['count', 'min', 'max'],
        'lat': ['mean', 'std'],
        'lon': ['mean', 'std']
    }).round(4)
    
    taxi_stats.columns = ['点数', '最早时间', '最晚时间', 
                         '平均纬度', '纬度标准差', '平均经度', '经度标准差']
    
    print(f"出租车数量: {len(taxi_stats)}")
    print("\n出租车活动统计:")
    display(taxi_stats.describe())
    
# 绘制出租车活动分布
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # 点数分布
    ax1 = axes[0]
    taxi_counts = taxi_stats['点数'].sort_values()
    ax1.hist(taxi_counts, bins=30, color='skyblue', alpha=0.7, edgecolor='black')
    ax1.set_xlabel('每个出租车的点数')
    ax1.set_ylabel('出租车数量')
    ax1.set_title('出租车活动量分布')
    ax1.axvline(taxi_counts.mean(), color='red', linestyle='--', 
                label=f'均值: {taxi_counts.mean():.1f}')
    ax1.legend()
    
    # 累积分布
    ax2 = axes[1]
    taxi_counts_sorted = taxi_counts.sort_values(ascending=False).reset_index(drop=True)
    cumulative_percent = (taxi_counts_sorted.cumsum() / taxi_counts_sorted.sum()) * 100
    
    ax2.plot(range(1, len(cumulative_percent) + 1), cumulative_percent, 
             color='green', linewidth=2)
    ax2.set_xlabel('出租车排名')
    ax2.set_ylabel('累积活动量占比 (%)')
    ax2.set_title('出租车活动量累积分布')
    ax2.grid(True, alpha=0.3)
    
    # 标记关键点
    for percent in [20, 50, 80]:
        idx = np.argmax(cumulative_percent >= percent)
        ax2.scatter(idx + 1, cumulative_percent[idx], color='red', s=50)
        ax2.text(idx + 1, cumulative_percent[idx] + 2, 
                f'{percent}%: 前{idx + 1}辆出租车', fontsize=9)
    
    plt.tight_layout()
    plt.show()

## 7. 轨迹数据处理

In [ ]:
# 加载完整轨迹数据
print("加载轨迹数据（采样率: 0.5%）...")

try:
    # 使用项目的数据加载器加载轨迹
    trajectories = loader.load_all_trajectories(
        sample_rate=0.005,  # 0.5% 采样率
        limit_per_file=50   # 每个文件最多50条轨迹
    )
    
    print(f"成功加载 {len(trajectories)} 条轨迹")
    
    # 显示第一条轨迹的信息
    if trajectories:
        first_traj = trajectories[0]
        print(f"\n第一条轨迹信息:")
        print(f"  出租车ID: {first_traj.taxi_id}")
        print(f"  点数: {len(first_traj.points)}")
        print(f"  开始时间: {first_traj.start_time}")
        print(f"  结束时间: {first_traj.end_time}")
        print(f"  持续时间: {first_traj.duration/3600:.2f} 小时")
        print(f"  总距离: {first_traj.total_distance/1000:.2f} km")
        print(f"  平均速度: {first_traj.avg_speed*3.6:.2f} km/h")
        
except Exception as e:
    print(f"加载轨迹数据时出错: {e}")

In [ ]:
# 预处理轨迹数据
if 'trajectories' in locals() and trajectories:
    print("\n预处理轨迹数据...")
    
    preprocessor = TrajectoryPreprocessor(
        min_trajectory_length=10,
        max_speed_kmh=120.0,
        sampling_interval_s=60.0,
        remove_outliers=True
    )
    
    processed_trajectories = preprocessor.preprocess_trajectories(trajectories)
    print(f"预处理后保留 {len(processed_trajectories)} 条轨迹 "
          f"({len(processed_trajectories)/len(trajectories)*100:.1f}%)")
    
    # 显示预处理前后的对比
    if processed_trajectories:
        processed_traj = processed_trajectories[0]
        print(f"\n预处理后的第一条轨迹信息:")
        print(f"  点数: {len(processed_traj.points)}")
        print(f"  持续时间: {processed_traj.duration/3600:.2f} 小时")
        print(f"  总距离: {processed_traj.total_distance/1000:.2f} km")
        print(f"  平均速度: {processed_traj.avg_speed*3.6:.2f} km/h")

In [ ]:
# 可视化轨迹
if 'processed_trajectories' in locals() and processed_trajectories:
    # 选择前10条轨迹进行可视化
    trajectories_to_plot = processed_trajectories[:10]
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # 1. 所有轨迹叠加图
    ax1 = axes[0, 0]
    colors = plt.cm.tab10(np.linspace(0, 1, len(trajectories_to_plot)))
    
    for i, traj in enumerate(trajectories_to_plot):
        points_array = traj.to_points_array()
        if len(points_array) > 1:
            ax1.plot(points_array[:, 1], points_array[:, 0], 
                    color=colors[i], alpha=0.7, linewidth=1, 
                    label=f'Taxi {traj.taxi_id[:8]}')
    
    ax1.set_xlabel('经度')
    ax1.set_ylabel('纬度')
    ax1.set_title('出租车轨迹叠加图')
    ax1.legend(loc='upper right', fontsize=8)
    ax1.grid(True, alpha=0.3)
    
    # 2. 轨迹点密度图
    ax2 = axes[0, 1]
    all_points = []
    for traj in trajectories_to_plot:
        points_array = traj.to_points_array()
        all_points.extend(points_array.tolist())
    
    all_points = np.array(all_points)
    if len(all_points) > 0:
        hb = ax2.hexbin(all_points[:, 1], all_points[:, 0], 
                       gridsize=30, cmap='YlOrRd', alpha=0.8)
        ax2.set_xlabel('经度')
        ax2.set_ylabel('纬度')
        ax2.set_title('轨迹点密度热力图')
        plt.colorbar(hb, ax=ax2, label='点数')
    
    # 3. 轨迹长度分布
    ax3 = axes[1, 0]
    distances = [traj.total_distance/1000 for traj in trajectories_to_plot]  # km
    durations = [traj.duration/3600 for traj in trajectories_to_plot]  # hours
    
    scatter = ax3.scatter(durations, distances, c=range(len(trajectories_to_plot)), 
                         cmap='viridis', s=100, alpha=0.7, edgecolors='black')
    ax3.set_xlabel('持续时间 (小时)')
    ax3.set_ylabel('总距离 (km)')
    ax3.set_title('轨迹长度与持续时间关系')
    plt.colorbar(scatter, ax=ax3, label='轨迹序号')
    
    # 添加平均速度线
    speeds = [dist/dur if dur > 0 else 0 for dist, dur in zip(distances, durations)]
    avg_speed = np.mean([s for s in speeds if s > 0])
    
    x_max = max(durations) * 1.1
    y_line = avg_speed * x_max
    ax3.plot([0, x_max], [0, y_line], 'r--', alpha=0.5, 
            label=f'平均速度: {avg_speed:.1f} km/h')
    ax3.legend()
    
    # 4. 轨迹速度分布
    ax4 = axes[1, 1]
    avg_speeds = []
    max_speeds = []
    
    for traj in trajectories_to_plot:
        if hasattr(traj, 'avg_speed') and traj.avg_speed:
            avg_speeds.append(traj.avg_speed * 3.6)  # 转换为km/h
            max_speeds.append(traj.max_speed * 3.6)  # 转换为km/h
    
    if avg_speeds:
        x_pos = range(len(avg_speeds))
        width = 0.35
        
        ax4.bar([p - width/2 for p in x_pos], avg_speeds, width, 
               color='skyblue', alpha=0.7, label='平均速度')
        ax4.bar([p + width/2 for p in x_pos], max_speeds, width, 
               color='salmon', alpha=0.7, label='最大速度')
        
        ax4.set_xlabel('轨迹序号')
        ax4.set_ylabel('速度 (km/h)')
        ax4.set_title('轨迹速度分布')
        ax4.set_xticks(x_pos)
        ax4.set_xticklabels([f'T{i+1}' for i in x_pos])
        ax4.legend()
    
    plt.suptitle('出租车轨迹分析', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 8. 数据探索总结

In [ ]:
# 生成数据探索总结
print("数据探索分析总结")
print("=" * 60)

summary = {}

if 'df_sample' in locals():
    summary["样本数据"] = {
        "总行数": len(df_sample),
        "出租车数量": df_sample['taxi_id'].nunique(),
        "时间范围": f"{df_sample['timestamp'].min()} 到 {df_sample['timestamp'].max()}",
        "纬度范围": f"{df_sample['lat'].min():.4f} 到 {df_sample['lat'].max():.4f}",
        "经度范围": f"{df_sample['lon'].min():.4f} 到 {df_sample['lon'].max():.4f}",
        "缺失值比例": f"{df_sample.isnull().sum().sum() / (len(df_sample) * len(df_sample.columns)) * 100:.2f}%"
    }

if 'trajectories' in locals():
    summary["轨迹数据"] = {
        "轨迹数量": len(trajectories),
        "平均轨迹长度": f"{np.mean([len(t.points) for t in trajectories]):.1f} 点",
        "平均持续时间": f"{np.mean([t.duration for t in trajectories])/3600:.2f} 小时",
        "平均距离": f"{np.mean([t.total_distance for t in trajectories])/1000:.2f} km"
    }

if 'processed_trajectories' in locals():
    summary["预处理后轨迹"] = {
        "保留轨迹比例": f"{len(processed_trajectories)/len(trajectories)*100:.1f}%",
        "平均速度": f"{np.mean([t.avg_speed*3.6 for t in processed_trajectories]):.1f} km/h",
        "最大速度": f"{np.max([t.max_speed*3.6 for t in processed_trajectories]):.1f} km/h"
    }

# 打印总结
for category, stats in summary.items():
    print(f"\n{category}:")
    print("-" * 40)
    for key, value in stats.items():
        print(f"  {key}: {value}")

In [ ]:
# 为聚类准备数据
print("\n聚类数据准备:")
print("-" * 40)

if 'processed_trajectories' in locals() and processed_trajectories:
    # 提取所有轨迹点
    all_points = []
    for traj in processed_trajectories:
        for point in traj.points:
            all_points.append([point.latitude, point.longitude])
    
    points_array = np.array(all_points)
    
    print(f"总点数: {len(points_array)}")
    print(f"数据形状: {points_array.shape}")
    print(f"纬度范围: [{points_array[:, 0].min():.4f}, {points_array[:, 0].max():.4f}]")
    print(f"经度范围: [{points_array[:, 1].min():.4f}, {points_array[:, 1].max():.4f}]")
    
    # 计算点密度
    lat_range = points_array[:, 0].max() - points_array[:, 0].min()
    lon_range = points_array[:, 1].max() - points_array[:, 1].min()
    area = lat_range * lon_range
    density = len(points_array) / area if area > 0 else 0
    
    print(f"空间面积: {area:.6f} 度²")
    print(f"点密度: {density:.1f} 点/度²")
    
    # 保存为聚类数据
    clustering_data_path = "../data/processed/clustering_points.npy"
    os.makedirs(os.path.dirname(clustering_data_path), exist_ok=True)
    np.save(clustering_data_path, points_array)
    print(f"\n聚类数据已保存到: {clustering_data_path}")
else:
    print("没有可用的预处理轨迹数据")

## 9. 结论与建议

### 关键发现

1. **数据质量**: 数据基本完整，缺失值较少，但有部分点在北京范围外，需要过滤
2. **空间分布**: 轨迹点集中在北京城区，有明显的热点区域
3. **时间特征**: 有明显的早晚高峰模式，工作日与周末模式不同
4. **出租车活动**: 不同出租车的活动量差异较大，少数出租车贡献了大部分数据
5. **轨迹特征**: 轨迹长度、持续时间和速度分布符合实际交通模式

### 对DBSCAN聚类的建议

1. **参数选择**:
   - eps: 建议在0.001-0.01之间进行测试
   - min_samples: 建议在5-15之间，根据点密度调整
   
2. **预处理策略**:
   - 过滤北京范围外的点
   - 去除异常速度的点
   - 对超长轨迹进行分段
   
3. **性能优化**:
   - 考虑使用空间索引加速邻域查询
   - 对于大规模数据，采用分批处理策略
   - 利用多核CPU进行并行计算

4. **聚类目标**:
   - 识别热点区域（如商业区、交通枢纽）
   - 发现出租车活动模式
   - 分析时空聚类特征